In [1]:
%cd ../
import pandas as pd
from torch.utils.data import Dataset,DataLoader
import torch
import numpy as np
import ast
import torch.nn as nn
import torch.optim as optim
import random
from matplotlib import pyplot as plt
import os
import shutil
from datetime import datetime
import wandb
from utils import set_all_seed, load_exp1_dataset
from mmoe import MMoE
from trainer_and_evaluator import train_MMoE, eval_MMoE

C:\jupyter file\CMU_3\distributed ML\project



In [2]:
set_all_seed(42)
train_dataset, val_dataset, test_dataset = load_exp1_dataset()

Absolute Pearson correlation coefficient: 0.17716269850240274
P-value: 0.0


In [3]:
# 设定各种超参数，wandb日志名，日志存储路径，模型存储路径
train_params = {
    "batch_size": 256,
    "N_epochs": 50,
    "lr": 0.0001
}

model_params={
    'feature_dim': 114,
    'expert_dim': 32,
    'n_expert': 4,
    'n_task': 2,
    'use_gate': True,
    'gate_dropout': 0,
    'tower_dropout': 0,
    'expert_dropout': 0
}

model_name="exp1_MMoE"
if not os.path.exists("model/"+model_name):
    os.makedirs("model/"+model_name) 

train_params_str = "_".join(f"{key}={value}" for key, value in train_params.items())
model_params_str = "_".join(f"{key}={value}" for key, value in model_params.items())
short_model_params_str = "_".join(f"{value}" for key, value in model_params.items())

wandb_name=model_name+":"+train_params_str+"_"+model_params_str

# 使用short_model_params_str是因为windows支持的最长文件名长度仅为260
bestmodel_save_dir=f"model/"+model_name+"/"+train_params_str+"_"+short_model_params_str 

print(wandb_name)
print(bestmodel_save_dir)

wandb.init(project="mmoe", name=wandb_name)

exp1_MMoE:batch_size=256_N_epochs=50_lr=0.0001_feature_dim=114_expert_dim=32_n_expert=4_n_task=2_use_gate=True_gate_dropout=0_tower_dropout=0_expert_dropout=0
model/exp1_MMoE/batch_size=256_N_epochs=50_lr=0.0001_114_32_4_2_True_0_0_0


wandb: Currently logged in as: yuntaozh (zhengyuntao). Use `wandb login --relogin` to force relogin


In [4]:
mymodel = MMoE(**model_params)

nParams = sum([p.nelement() for p in mymodel.parameters()])
print('Number of parameters: %d' % nParams)

nParams_in_mmoe=0
for name,p in mymodel.named_parameters():
    if name.startswith("Expert_Gate"):
        nParams_in_mmoe=nParams_in_mmoe+p.nelement()
print('Number of parameters in MMoE: %d' % nParams_in_mmoe)

Number of parameters: 11978
Number of parameters in MMoE: 9976


In [5]:
losses, val_losses, adam_batch_loss= train_MMoE(mymodel,
                                           train_dataset,
                                           val_dataset,
                                           bestmodel_save_dir,
                                           **train_params)

Epoch=0,train_loss=1.2273576259613037,val_loss=0.9586679339408875
current epoch is the best so far. Saving model...
Epoch=1,train_loss=0.9308820366859436,val_loss=0.9103913307189941
current epoch is the best so far. Saving model...
Epoch=2,train_loss=0.6807833313941956,val_loss=0.48898202180862427
current epoch is the best so far. Saving model...
Epoch=3,train_loss=0.4569131135940552,val_loss=0.4219781756401062
current epoch is the best so far. Saving model...
Epoch=4,train_loss=0.40758639574050903,val_loss=0.3834478259086609
current epoch is the best so far. Saving model...
Epoch=5,train_loss=0.36853867769241333,val_loss=0.35127025842666626
current epoch is the best so far. Saving model...
Epoch=6,train_loss=0.3430534899234772,val_loss=0.346282035112381
current epoch is the best so far. Saving model...
Epoch=7,train_loss=0.3287174105644226,val_loss=0.3263138234615326
current epoch is the best so far. Saving model...
Epoch=8,train_loss=0.3175148367881775,val_loss=0.3136332631111145
cur

In [6]:
# load best model based on validation
mybestmodel = MMoE(**model_params)
mybestmodel.load_state_dict(torch.load(bestmodel_save_dir))

<All keys matched successfully>

In [7]:
auc1, auc2=eval_MMoE(mybestmodel, test_dataset)

AUC: 0.9353350593868078
AUC: 0.9876592624197151


In [8]:
wandb.finish()

task1_loss,█▅▅▄▃▃▂▃▁▁▃▂▂▃▃▃▂▃▂▃▂▃▂▃▄▂▃▂▂▂▃▃▂▂▄▃▂▂▂▂
task2_loss,██▃▃▂▂▂▂▂▂▂▂▁▁▂▁▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▂▁▂▂▂
task_0/expert_0_weight,▁▁▁▁▁▁▁▂▁▁▁█▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁
task_0/expert_1_weight,▁▁▁▁▁▁▁▃▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
task_0/expert_2_weight,█▅█▅▅▄▂▃▂▃▃▄▃▂▂▃▂▂▂▂▄▂▂▂▂▂▂▂▂▃▃▂▂▁▂▃▂▂▂▂
task_0/expert_3_weight,▁▄▁▄▄▅▇▆▇▆▆▅▆▇▇▆▇▇▇▇▅▇▇▇▇▇▇▇▇▆▆▇▇█▇▆▇▇▇▇
task_1/expert_0_weight,▁▁▆▇▇▇▇▆▇▆▇▇█████▇█▇▇▇▇▇██▇▇▇▇█▇████▇███
task_1/expert_1_weight,▁▁█▇▇▇██▇█▇▇▇▇▇▇▇█▇▇▇▇█▇▆▆▇▇▇▆▆▇▇▇▇▆▇▇▆▆
task_1/expert_2_weight,██▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
task_1/expert_3_weight,▁▁▅▁▁▁▁▁█▅▅▅▁▁▁▁▅▁▅▁▅█▅▆▂▁▁▄▂▁▁▁▅▁▅▁█▁█▁
train_loss,█▆▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
